<a href="https://colab.research.google.com/github/WAPunsisiYemaniPerera/Integration-of-LLMs-using-Langchain-0.3.27/blob/main/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up and installation

## Step 1: Install Required Packages

In [1]:
# Install LangChain 0.3.27 and dependencies
!pip install langchain==0.3.27
!pip install langchain-openai
!pip install python-dotenv

In [2]:
# Install additional packages for document processing
!pip install pypdf2
!pip install chromadb
!pip install tiktoken

In [3]:
# install packages for web scraping and utilities
!pip install beautifulsoup4
!pip install requests
!pip install pandas numpy

In [4]:
!pip install langchain-community

### Step 2: Import Required Libraries

In [5]:
# This setup lets you load text, split it,
# store conversation context, and run it through an
# OpenAI-powered chat chain with custom prompts.
import os
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

## Basic Configuration

### Step 3: Configure OpenRouter

First, you need to get an API key from [OpenRouter](https://openrouter.ai/). Then set it up in Colab:


In [6]:
# set the openrouter API key
import getpass
api_key = getpass.getpass("Enter your open Router Key: ")
os.environ["OPENAI_API_KEY"]=api_key

Enter your open Router Key: ··········


In [7]:
# configure the LLM with open router
llm = ChatOpenAI(
    base_url = "https://openrouter.ai/api/v1",
    model = "openai/gpt-oss-20b:free",
    temperature= 0.7
)

Low temperature (e.g., 0–0.3) → More deterministic and focused answers. The model sticks closely to facts or common responses.

Medium temperature (e.g., 0.5–0.7) → Balances creativity and reliability. Responses are varied but still make sense.

High temperature (e.g., 0.8–1.0) → Very creative and unpredictable. Responses can be imaginative, but may include odd or less accurate outputs.

✅ Analogy: Think of it as a “creativity knob.” Low = safe & precise, High = wild & creative.

### Step 4: Test Basic Connection

In [8]:
# Test the connection
response = llm.invoke([HumanMessage(content="Hello! can youconfirm the connection is working?")])
print(response.content)

analysisUser: Hello! can you confirm the connection is working? They want confirmation that the connection is working. We should respond politely.assistantfinalHello! Yes, the connection is working—happy to help with whatever you need.


## Scenario 1: Customer Support Chatbot

### Real-World Application: E-commerce Customer Service

Let's build a customer support chatbot for an online electronics store.

In [10]:
# Create a system prompt for customer support
system_prompt = """You are a helpful customer support agent for Vet Store,
an online pet items retailer. You should be friendly, professional,
and knowledgeable about common issues like orders, returns, and products."""

Defines the system message for the LLM.

Tells the model its role: a friendly, professional customer support agent.

Gives context about the domain:

In [16]:
# Create a conversation memory
memory = ConversationBufferMemory(return_messages=True)

Creates a conversation memory:

Stores the full chat history so the model remembers previous messages.

return_messages=True ensures the memory returns all messages when queried.

In [18]:
# Create the customer support chain
support_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])


ChatPromptTemplate.from_messages([...]) → Creates a prompt template for a chat model.

("system", system_prompt) → Inserts the system message you defined earlier (role and instructions).

("memory", "{input}") → Placeholder for user input that will be fed into the chat chain.

In [20]:
from langchain_core.runnables import RunnableSequence
from operator import itemgetter

# Create the customer support chain with memeory integration
# This pattern loads memory, formats the prompt, invokes the LLM,and save the context
support_chain = (
    {
        "input": itemgetter("input"),
        "chat_history": lambda x: memory.load_memory_variables({})["history"]

    }
    | support_prompt
    | llm
)

itemgetter("input") → Extracts the user input from a dictionary. ✅

lambda x: memory.load_memory_variables({})["history"] → Loads chat history from memory. ✅

support_prompt → Formats the messages into a prompt. ✅

llm → Sends the formatted prompt to the model. ✅

Memory in LangChain doesn’t automatically store messages just by creating the prompt.

When you run the chain (i.e., give it user input and get a response), the chain can save both the user input and the model’s output back to memory.

So, the comment is just a reminder: don’t forget to update memory after each interaction if you’re managing it manually.

In [21]:
# Test the customer support bot
response = support_chain.invoke({"input": "I ordered a dog food 2 days ago but haven't recieved the item"})
print(response.content)

analysisUser: "I ordered a dog food 2 days ago but haven't recieved the item". They need support regarding missing order. We need to respond friendly, ask for order number, check status, maybe mention shipping times, advise next steps.assistantfinalHi there! I’m sorry to hear that your dog food hasn’t arrived yet. Let’s get this sorted out as quickly as possible.

Could you please share your order number or the email address you used to place the order? Once I have that, I can:

1. **Check the shipment status** – see where it is in transit and the expected delivery date.  
2. **Confirm the shipping address** – sometimes a small typo can delay delivery.  
3. **Arrange a replacement or refund** if the package is confirmed lost.

In the meantime, most of our orders typically arrive within 3–5 business days, depending on your location. If it’s been only two days, it’s a good idea to double‑check the shipping carrier’s tracking link (you’ll find it in your confirmation email).

Looking forw

support_chain.invoke({...}) → Sends the user input to the chain.

The chain will:

Load memory (chat history),

Format the prompt with the system instructions,

Call the LLM, and

Optionally save the conversation back to memory (if your chain/memory is set up correctly).

response.content → Retrieves the actual text output from the LLM.

### Advanced Customer Support Features

In [22]:
# Create a knowledge base for common issues
knowledge_base = {
    "shipping": "Standard shipping takes 3-5 business days. Express shipping takes 1-2 business days.",
    "returns": "You can return items within 30 days of purchase. Items must be in original condition.",
    "warranty": "All electronics come with a 1-year expiration date."
}

knowledge_base → A Python dictionary storing common customer support answers.

Each key is a topic (like "shipping", "returns", "warranty").

Each value is the corresponding information the bot can provide to users.

✅ In short: This acts as a mini FAQ that your customer support bot can reference for quick answers.

In [23]:
# Enhanced support chain with knowledge lookup
def enhanced_support_response(question):
  # simple keyword matching for demo
  for key, info in knowledge_base.items():
    if key.lower() in question.lower():
      return f"Based on our policy: {info}\n\nLet me provide more specific help with your question."
    return "Let me help you with that question."

enhanced_support_response(question) → A function that takes a user’s question as input.

Keyword matching loop:

Checks if any key from knowledge_base (like "shipping", "returns") appears in the user’s question.

If it finds a match, it returns the corresponding info with a friendly message.

Fallback:

If no keywords match, it returns a generic response: "Let me help you with that question."

In [25]:
# Test enhanced support
kb_info = enhanced_support_response("What is your return policy?")
#Calls your knowledge base function first.
#Returns a policy-based response if a keyword is matched.

full_response = support_chain.invoke({"input": f"{kb_info}\n\nCustomer question: What's your return policy?"})
print(full_response.content)
#Sends the knowledge-augmented input to your LLM chain.
#The LLM now sees both the knowledge base info and
#the original question, so it can generate a friendly,
#conversational answer.

analysisWe need to answer the question: "What's your return policy?" The user says "Let me help you with that question." That seems like they are telling us they will help? Actually the user says "Let me help you with that question." Then "Customer question: What's your return policy?" So they are asking us to answer the return policy question. We need to provide a friendly, professional, knowledgeable answer about the return policy. We should say something like: 30 days, condition of product, etc. Since we don't have actual policy details, we can provide generic standard policy: 30 days to return, original packaging, refund or exchange, etc. Also mention shipping costs, restocking fee, etc. We'll use friendly tone, mention steps to return. We'll also ask if they need more help.assistantfinalSure thing!  
At Vet Store we want you and your furry friend to be completely happy with every purchase. Here’s a quick rundown of our return policy:

| **Aspect** | **Details** |
|------------|---